<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">François Rechenmann &amp; Thierry Parmentelat&nbsp;<img src="media/inria-25.png" style="display:inline"></span><br/>

# Comptage des nucléotides sur une fenêtre

Dans ce complément nous allons écrire un programme python permettant de visualiser les comptages de nucléotides, sur des fenêtres glissantes et recouvrantes, comme cela a été expliqué dans la vidéo.

Commençons comme toujours par la cellule de compatibilité python2/python3

In [ ]:
# la formule magique pour utiliser print() en python2 et python3
from __future__ import print_function
# pour que la division se comporte en python2 comme en python3
from __future__ import division

Et de même nous allons avoir besoin de `matplotlib` pour dessiner les résultats sous forme de graphiques&nbsp;:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as pyplot

# la taille à utiliser pour les figures
import pylab
pylab.rcParams['figure.figsize'] = 10., 6.

### Comptage sur un morceau d'ADN

Dans le premier algorithme que nous avions écrit en python, nous comptions les fréquences des 4 bases dans **tout un brin** d'ADN. Dans le contexte présent, cela a besoin d'être un peu amélioré de façon à ne considérer qu'un **morceau** de la chaine d'entrée. 

Aussi, commençons par quelques notions de python qui nous seront utiles.

### Les indices en python

Pour accéder à un caractère dans la chaine à partir de son indice, en python la syntaxe est comme toujours très simple, on utilise simplement des crochets - c'est comme dans le pseudo-langage utilisé dans les vidéos.

Par contre il nous faut faire un peu plus attention car en python, contrairement cette fois aux hypothèses de la vidéo, **les indices commencent à zéro**. Mais pas de panique, ça reste très simple&nbsp;:

In [ ]:
string = "abc"
print("à l'indice 0:", string[0])
print("à l'indice 1:", string[1])
print("à l'indice 2:", string[2])

### Le *slicing* en python

Il existe aussi un trait moins habituel - qu'on appelle le *slicing* - qui est la capacité de construire des *tranches* à partir d'une séquence, avec la notation `[debut:fin]`. Commençons par un exemple simple&nbsp;:

In [ ]:
string = "abcdefghijklmnopqrstuv"
zoom = string[3:6]
print(zoom)

Pour lever toute ambigüité concernant les poteaux, voyez qu'on n'a pas besoin de faire de gymnastique compliquée&nbsp;:

In [ ]:
string[0:3]

In [ ]:
string[3:6]

In [ ]:
string[6:9]

Et on peut même tirer parti d'une astuce très pratique, c'est que si on donne une borne droite supérieure à la taille du tableau, ce n'est pas grave&nbsp;:

In [ ]:
string[9:200]

### Reprenons

Avec toutes ces nouvelles armes à notre disposition, nous pouvons améliorer notre fonction de comptage; pour ne compter que dans un morceau de l'ADN délimité entre les indices `debut` et `fin`, on peut faire tout simplement comme ceci&nbsp;:

In [ ]:
def compter_c_g(adn, debut, fin):
    # les valeurs de retour
    c = g = 0
    # on ne fait le parcours que sur un morceau
    for nucleo in adn[debut:fin]:
        if nucleo == 'C':
            c += 1
        elif nucleo == 'G':
            g += 1
    # on retourne les deux résultats
    return c, g

### Les fenêtres glissantes

Grâce à cette fonction de comptage, nous allons pouvoir écrire l'algorithme qui nous occupe. 

Comme dans le cas de la promenade, nous avons besoin de calculer deux listes, correspondant aux abscisses et aux ordonnées de la courbe qu'on veut dessiner. Naturellement ici nous allons choisir&nbsp;:
 * de représenter en abscisse l'indice du début de la fenêtre de calcul. On pourrait tout aussi bien choisir la fin ou le milieu, ça ne ferait que de translater la courbe. 
 * de représenter en ordonnée le taux $\frac{G-C}{G+C}$

Enfin avant de nous lancer, remarquons qu'il est nécessaire d'être prudent, car dans le cas - improbable, mais pas totalement impossible - où une fenêtre n'aurait **aucun `C` et aucun `G`**, on **ne peut pas diviser par $C+G=0$**. Aussi dans ces cas-là on choisit de considérer que le taux a une valeur nulle.

Tout ceci nous donne le code suivant&nbsp;:

In [ ]:
def fenetre_x_y(adn, fenetre, recouvrement):
    """
 en entrée
    adn:          le brin d'ADN
    fenetre:      la largeur de la fenêtre
    recouvrement: de combien se recouvrent deux fenêtres consécutives
 en sortie
    X:            liste des abscisses - les multiples de (fenetre-recouvrement)
    Y:            liste des ordonnees - les valeurs de (G-C)/(G+C)
    """

    # la longueur de l'adn - pour ne pas avoir
    # à la recalculer à chaque boucle
    longueur = len(adn)
    # le début de la fenêtre
    debut = 0
    # les deux listes de résultats
    X = []
    Y = []
    
    while debut < longueur:
        # avec le slicing ce n'est pas grave si on déborde à droite
        c, g = compter_c_g (adn, debut, debut+fenetre)
        # dans tous les cas, en abscisse on prend debut
        x = debut
        # le cas pathologique où on n'a aucun C ou G
        if c == 0 and g == 0:
            y = 0.
        else:
            y = (g - c) / (g + c)
        # on range ce point dans les résultats
        X.append(x)
        Y.append(y)
        # on n'oublie pas de décaler la fenêtre 
        debut += (fenetre - recouvrement)

    # c'est fini on peut retourner le resultat
    return X, Y

### Raccourci

Comme nous l'avions fait pour la promenade, nous allons nous définir un raccourci pour calculer et afficher ce résultat en un seul appel. Pour améliorer la lisibilité, nous allons en profiter pour tracer une ligne rouge qui matérialise les ordonnées nulles&nbsp;:

In [ ]:
def fenetre_glissante(adn, fenetre, recouvrement):
    X, Y = fenetre_x_y(adn, fenetre, recouvrement)
    pyplot.plot(X, Y)
    # on trace un trait correspondant à y=0 sur toute la largeur
    # qui est obtenue comme X[-1], c'est-à-dire le dernier élément de X
    pyplot.plot([0, X[-1]], [0, 0], color='r', linestyle='dashed', linewidth=2)
    pyplot.show()

### Sur des entrées de test

Avant de lancer cet outil sur des séquences réelles, il peut être utile de se convaincre qu'il calcule bien ce qui est attendu sur des cas où on peut facilement faire les calculs de tête, comme celui-ci&nbsp;:

In [ ]:
test = 3* (5*'C' + 5*'G')
print(test)

In [ ]:
fenetre_glissante(test, 10, 5)

On obtient bien ici une valeur nulle pour toutes les fenêtres non-tronquées, puisque si on prend un fragment de 10 lettres consécutives on a toujours 5 `C` et 5 `G`. La dernière fenêtre par contre, qui ne contient que les 5 dernieres lettres, donne bien un taux de 100% de `G`.

Je vous invite à titre d'exercice à vérifier que le résultat reste correct si on modifie la largeur du recouvrement&nbsp;:

In [ ]:
fenetre_glissante(test, 10, 3)

### Des données réelles

Prenons par exemple l'échantillon de Borrelia sur lequel nous avions observé un point de rebroussement assez net&nbsp;:

In [ ]:
# l'échantillon de Borrelia de la séquence 7 sur la promenade
from samples import borrelia
print("longueur", len(borrelia))

Vous allez voir que cette technique de dessin nous permet aussi de soupçonner ce rebroussement&nbsp;:

In [ ]:
fenetre_glissante(borrelia, 400, 100)

Et ceci même à des échelles plus grossières. Voici ce que nous avons obtenu par exemple&nbsp;:

![](media/fenetre-borrelia.png)

Nous vous invitons à essayer diverses valeurs des paramètres pour vous approcher de ce rendu&nbsp;:

In [ ]:
fenetre_glissante(borrelia, 2000, 500)

***

### Ou encore depuis ENA

En option, voici un squelette pour vous permettre d'exécuter notre algorithme sur la séquence de votre choix dans ENA. À vous de choisir une clé, et d'ajuster les paramètres de `fenetre_glissante` en fonction de la longueur.

In [ ]:
import fetch

In [ ]:
from_ena = fetch.fetch('AE000789')
print("longueur", len(from_ena))

In [ ]:
fenetre_glissante(from_ena, 300, 100)